In [1]:
! pip install pyspark

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import ClusteringEvaluator

In [ ]:
# conectado ao drive através da interface gráfica
'''
from google.colab import drive
drive.mount('/content/drive')
'''

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
pagamentos_pedido_preprocessados = spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/pagamentos_pedido_preprocessados_parquet')
join_itens_pedido = spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/join_itens_pedido_parquet')
produtos_preprocessados = spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/produtos_preprocessados_parquet')
avaliacoes_pedido = spark.read.option('header', 'true').csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/avaliacoes_pedido.csv')
pedidos = spark.read.option('header', 'true').csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv')

In [6]:
pagamentos_pedido_preprocessados.show(n=5, truncate=False)
join_itens_pedido.show(n=5, truncate=False)
produtos_preprocessados.show(n=5, truncate=False)
avaliacoes_pedido.show(n=5, truncate=False)
pedidos.show(n=5, truncate=False)

+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|id_pedido                       |sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|valor_pagamento_normalizado|parcelas_pagamento_normalizado|
+--------------------------------+-------------------+--------------+------------------+---------------+---------------------------+------------------------------+
|b81ef226f3fe1789b1e8b2acac839d17|1                  |credit_card   |8                 |99.33          |0.007269424652080491       |0.3333333333333333            |
|a9810da82917af2d9aefd1278f1dcfa0|1                  |credit_card   |1                 |24.39          |0.0017849719849415402      |0.041666666666666664          |
|25e8ea4e93396b6fa0d3dd708e76c1bd|1                  |credit_card   |1                 |65.71          |0.004808958963940492       |0.041666666666666664          |
|ba78997921bbcdc

In [7]:
itens_pedido_df = join_itens_pedido.na.drop()

montar_vetor = VectorAssembler(
    inputCols=['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm'],
    outputCol='caracteristicas'
)

itens_pedido_df = montar_vetor.transform(itens_pedido_df)

itens_pedido_df.show(n=5, truncate=False)

+--------------------------------+--------------------------------+--------------+--------------------------------+-------------------+-----+-----------+--------------------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+------------------------+
|id_produto                      |id_pedido                       |item_id_pedido|id_vendedor                     |data_limite_envio  |preco|valor_frete|id_cliente                      |status_pedido|data_compra_pedido |data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|categoria_produto    |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|caracteristica

In [8]:
kmeans = KMeans(featuresCol='caracteristicas', k=3)
modelo_kmeans = kmeans.fit(itens_pedido_df)

In [9]:
predicao = modelo_kmeans.transform(itens_pedido_df)
predicao.select('id_pedido', 'prediction').show(n=5, truncate=False)

+--------------------------------+----------+
|id_pedido                       |prediction|
+--------------------------------+----------+
|00010242fe8c5a6d1ba2dd792cb16214|0         |
|00048cc3ae777c65dbb7d2a0634bc1ea|0         |
|000576fe39319847cbb9d288c5617fa6|1         |
|00061f2a7bc09da83e415a52dc8a4af1|0         |
|0009c9a17f916a706d71784483a5d643|0         |
+--------------------------------+----------+
only showing top 5 rows



In [10]:
evaluator = ClusteringEvaluator(featuresCol='caracteristicas', predictionCol='prediction', metricName='silhouette')
silhouette = evaluator.evaluate(predicao)
print(f'Silhouette Score: {silhouette}')

Silhouette Score: 0.91844438474722


In [13]:
# predição com os dados normalizados:
scaler = MinMaxScaler(inputCol='caracteristicas', outputCol='caracteristicas_normalizado')
modelo_scaler = scaler.fit(itens_pedido_df)
itens_pedido_normalizado_df = modelo_scaler.transform(itens_pedido_df)

kmeans = KMeans(featuresCol='caracteristicas_normalizado', k=3)
modelo_kmeans = kmeans.fit(itens_pedido_normalizado_df)

predicao_normalizada = modelo_kmeans.transform(itens_pedido_normalizado_df)
predicao_normalizada.select('id_pedido', 'prediction').show(n=5, truncate=False)

evaluator = ClusteringEvaluator(featuresCol='caracteristicas_normalizado', predictionCol='prediction', metricName='silhouette')
silhouette = evaluator.evaluate(predicao_normalizada)
print(f'Silhouette Score: {silhouette}')

+--------------------------------+----------+
|id_pedido                       |prediction|
+--------------------------------+----------+
|00010242fe8c5a6d1ba2dd792cb16214|0         |
|00048cc3ae777c65dbb7d2a0634bc1ea|0         |
|000576fe39319847cbb9d288c5617fa6|2         |
|00061f2a7bc09da83e415a52dc8a4af1|0         |
|0009c9a17f916a706d71784483a5d643|0         |
+--------------------------------+----------+
only showing top 5 rows

Silhouette Score: 0.634664688301955


In [14]:
spark.stop()